In [1]:
import sopa
import spatialdata
import pandas as pd
import scanpy as sc
import numpy as np

###PAREMETERSawddwasdwa

proportion_constant = 1
Tissue = "Region2"
TableFolder = "REGION2_TABLES"

sdata = spatialdata.read_zarr(f"/Volumes/ProstateCancerEvoMain/dbs/Ongoing/{Tissue}/{Tissue}_Xenium_Phen_HE_Integrated.V4.ongoing.zarr")
sdata

adata = sc.read_h5ad(f"/Volumes/ProstateCancerEvoMain/dbs/Ongoing/{Tissue}/{TableFolder}/{Tissue}_Xenium_Phen_HE_Integrated.GeneTranscripts_XStock_Native.V1.h5ad")
new_ids = adata.obs["cell_id"].tolist()
shape = sdata.shapes['cell_boundaries']

/Users/ugursahin/miniforge3/envs/ScanPy/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/ugursahin/miniforge3/envs/ScanPy

In [2]:
import pandas as pd


records = []
# zip the new_ids to the geometry items (ensure they’re in the same order!)
for new_id, (old_id, poly) in zip(new_ids, shape['geometry'].items()):
    for vi, (x, y) in enumerate(poly.exterior.coords, start=1):
        records.append({
            'cell_id': new_id,
            'vertex':  vi,
            'x':       x,
            'y':       y
        })

shape_df = pd.DataFrame(records)
shape_df

,cell_id,vertex,x,y
0,aaaaealn-1,1,477.912506,4855.412598
1,aaaaealn-1,2,476.212524,4857.325195
2,aaaaealn-1,3,474.937500,4858.175293
3,aaaaealn-1,4,473.450012,4858.812500
4,aaaaealn-1,5,472.812500,4858.812500
...,...,...,...,...
7488830,oimbdlnd-1,15,10210.837891,4699.649902
7488831,oimbdlnd-1,16,10211.049805,4699.437500
7488832,oimbdlnd-1,17,10211.049805,4698.587402
7488833,oimbdlnd-1,18,10210.413086,4697.950195


In [3]:
#We just need it to obtain shape of image
C = 0
img = sdata.images['Xenium_Image']['scale0']['image'].isel(c=C)
arr    = np.asarray(img)


import numpy as np
from skimage.draw import polygon

#Fill with cell_id

H, W = arr.shape
mask = np.zeros((H, W), dtype=np.int32)

#proportion_constant = 4.7065

labels = {}
for label, (cell_id, grp) in enumerate(shape_df.groupby('cell_id'), start=1):
    labels[label] = cell_id
    cols = np.floor(grp['x'].values * proportion_constant).astype(int)
    rows = np.floor(grp['y'].values * proportion_constant).astype(int)

    # clip into valid pixel bounds
    cols = np.clip(cols, 0, W-1)
    rows = np.clip(rows, 0, H-1)

    # rasterize the polygon, filling those pixels with `label`
    rr, cc = polygon(rows, cols, shape=mask.shape)
    mask[rr, cc] = label


In [4]:
np.savetxt(f"{Tissue}_mask.csv", mask, fmt="%d", delimiter=",")
records_df = pd.DataFrame(records)
records_df.to_csv(f"{Tissue}_mask_labels.csv", header=False, index=False)

## Mask is prepared (the matrix defined the borders based on index), and it must be applied props function after that point.

In [7]:
import numpy as np
import pandas as pd
from skimage import data, filters, measure, morphology
import plotly.express as px
import plotly.graph_objects as go
import tifffile as tfi


#for i in range(0, 4):
for ie in range(0,2): #Temporarily calculate for channel 1 just for now !
    i = 0
    try:
        img_raw = sdata.images['Xenium_Image']['scale0']['image'].isel(c=i)
        #label_image = np.loadtxt("/Users/ugursahin/RAO/Tissue1_mask.csv", delimiter=",", dtype=int)
        label_image = mask
        
        img = np.asarray(img_raw)

        props = measure.regionprops_table(
            label_image,
            intensity_image=img,
            properties=['area',
        'area_bbox',
        'area_convex',
        'area_filled',
        'axis_major_length',
        'axis_minor_length',
        'centroid',
        'centroid_local',
        'centroid_weighted',
        'centroid_weighted_local',
        'coords',
        'eccentricity',
        'equivalent_diameter_area',
        'euler_number',
        'extent',
        'feret_diameter_max',
        'image',
        'image_convex',
        'image_filled',
        'image_intensity',
        'inertia_tensor',
        'inertia_tensor_eigvals',
        'intensity_max',
        'intensity_mean',
        'intensity_min',
        'label',
        'moments',
        'moments_central',
        'moments_hu',
        'moments_normalized',
        'moments_weighted',
        'moments_weighted_central',
        'moments_weighted_hu',
        'moments_weighted_normalized',
        'orientation',
        'perimeter',
        'perimeter_crofton',
        'slice',
        'solidity']
        )


        props_df = pd.DataFrame(props)
        props_df.to_csv(f"{Tissue}_ExtractedProps"+"_forChannel"+str(i)+".csv", header=True)
    except:
        print("Error in processing image for channel: ", i)
        continue


/Users/ugursahin/miniforge3/envs/ScanPy/lib/python3.11/site-packages/skimage/measure/_regionprops.py:694: RuntimeWarning: invalid value encountered in scalar divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim))
/Users/ugursahin/miniforge3/envs/ScanPy/lib/python3.11/site-packages/skimage/measure/_moments.py:326: RuntimeWarning: invalid value encountered in scalar divide
  nu[powers] = (mu[powers] / scale ** sum(powers)) / (


In [6]:
sdata

SpatialData object, with associated Zarr store: /Volumes/ProstateCancerEvoMain/dbs/Ongoing/Region2/Region2_Xenium_Phen_HE_Integrated.V4.ongoing.zarr
├── Images
│     ├── 'HE_Image': DataTree[cyx] (3, 14052, 23906), (3, 7026, 11953), (3, 3513, 5977), (3, 1757, 2989), (3, 878, 1494), (3, 439, 747)
│     ├── 'PhenoCycler_Image': DataTree[cyx] (25, 12563, 22914), (25, 6281, 11457), (25, 3140, 5728), (25, 1570, 2864), (25, 785, 1432)
│     └── 'Xenium_Image': DataTree[cyx] (4, 34103, 54036), (4, 17051, 27018), (4, 8525, 13509), (4, 4262, 6754), (4, 2131, 3377)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (299769, 1) (2D shapes)
│     ├── 'cell_boundaries_Backup': GeoDataFrame shape: (299769, 1) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (277431, 1) (2D shapes)
└── Tables
      ├── 'GeneTranscripts_XStockC': AnnData (299769, 5099)
      ├── 'GeneTranscripts_XStockP': AnnD